In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
MODEL= "gpt-4o-mini"
#MODEL= "mistral:7b"

In [40]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model= ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
    embeddings= OpenAIEmbeddings()
else:
    model= Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()


model.invoke("tell me a joke")   

AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-c96b8018-a291-4ef6-82d9-42b9e8bd65c1-0', usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [41]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
chain= model | parser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

In [42]:
from langchain_community.document_loaders import PyPDFLoader

loader= PyPDFLoader("domestico.pdf")
pages =loader.load_and_split()
pages


[Document(metadata={'source': 'domestico.pdf', 'page': 0}, page_content='(https://domesticolabora.com)\n\ue816\n15 años sirviendo\nEmpleadas de hogar\nSeleccionamos personal: bien formado, amable y eficiente\nLlámanos: 647 595 983(tel:+34647595983) \uf879\nEMPLEADAS DE HOGAR\n\uf236\nEmpleadas de hogar internas\nUno de los perfiles demandados por las familias es el de empleadas de hogar internas, es decir, aquellas empleadas que residen en el\ndomicilio del cliente durante los periodos de realización del trabajo. Las empleadas de hogar internas conviven con la familia, y\nrealizan las tareas de casa: limpieza de pisos o casas, plancha, cocina, cuidado de niños y personas mayores. En Domestico Labora,\nsabemos lo que supone el riesgo de dar la confianza a una trabajadora interna y por ello, formamos profesionalmente y seguimos de\nforma continua la evolución profesional y personal del servicio doméstico en su hogar.\n\uf64a\nEmpleadas de hogar externas\nCada día más, necesitamos ayuda p

In [43]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [44]:
chain= prompt | model | parser
chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})


"I don't know."

In [45]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, embedding=embeddings)

In [46]:
retriever = vectorstore.as_retriever()
retriever.invoke("Qué hace Domestico Labora?")

[Document(metadata={'source': 'domestico.pdf', 'page': 2}, page_content='Puedes realizar las entrevistas en nuestras instalaciones y conocer algunas o algunos candidatos,\npreviamente seleccionados por Doméstico Labora mediante un exigente protocolo de \x00ltro y\ncomprobaciones propio. De esta forma resulta muy e\x00ciente, discreto y protegido, y dispondrá de\nuna profesional experta que le ayudará en la entrevista y el análisis de cada caso. Sin ningún\ncompromiso! Más vale una imagen que mil palabras. Si lo pre\x00ere, puede conocer a los o las\ncandidatas seleccionadas en su domicilio, con información previa de la trayectoria de cada persona\ny un análisis posterior de resultados, dudas y valoraciones con una profesional de Doméstico\nLabora.\nNos gusta el trabajo bien hecho y los hogares bien cuidados.\nEmpleadas de hogar\nNuestro apoyo, seguimiento y formación es continuo si así lo necesita nuestro cliente. Nuestro objetivo; éxito a la primera,\nestabilidad a largo plazo, suavid

In [50]:
from operator import itemgetter
chain= (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    |prompt
    | model 
    | parser
)
print(chain.invoke({"question":"Como es el proceso de selección?"}))


El proceso de selección en Doméstico Labora incluye los siguientes pasos:

1. **Entrevista personal**: Se realiza una entrevista con el candidato.
2. **Estudio de currículum**: Se evalúa el currículum vitae del candidato.
3. **Experiencia y formación**: Se considera la experiencia previa y la formación del candidato.
4. **Prueba práctica**: Se lleva a cabo una prueba práctica de conocimientos y aptitudes.
5. **Comprobación de referencias**: Se verifican las referencias y resultados en trabajos anteriores.
6. **Valoración en equipo**: Se realiza una puesta en común y valoración en equipo de los candidatos.
7. **Puntuación y definición de campos de excelencia**: Se asigna una puntuación y se definen los campos en los que el candidato sobresale.
8. **Entrada en cartera selectiva**: Los candidatos seleccionados ingresan en una cartera selectiva para futuras oportunidades.

Este proceso está diseñado para asegurar la selección de personal bien formado, amable y eficiente. Además, Doméstico 